In [1238]:
import pandas as pd

In [1239]:
matches = pd.read_csv(r"C:\Users\melwi\Desktop\ML\project1\webscraping\pl_match_stats(2021,2022,2023).csv", index_col = 0)

In [1240]:
matches.head()

,season,date,day,time,round,team,opponent,venue,result,gf,ga,xg,xga,poss,attendance,captain,formation,opp formation,referee,sh,sot,dist,fk,pk,pkatt
0,2023,2023-08-11,Fri,20:00,Matchweek 1,Manchester City,Burnley,Away,W,3,0,1.9,0.3,65.0,21572.0,Kevin De Bruyne,4-2-3-1,5-4-1,Craig Pawson,17.0,8.0,13.9,0.0,0,0
1,2023,2023-08-19,Sat,20:00,Matchweek 2,Manchester City,Newcastle Utd,Home,W,1,0,1.0,0.3,59.0,53419.0,Kyle Walker,4-2-3-1,4-3-3,Robert Jones,14.0,4.0,17.9,0.0,0,0
2,2023,2023-08-27,Sun,14:00,Matchweek 3,Manchester City,Sheffield Utd,Away,W,2,1,3.5,0.7,79.0,31336.0,Kyle Walker,4-2-3-1,3-5-2,Jarred Gillett,29.0,9.0,17.3,2.0,0,1
3,2023,2023-09-02,Sat,15:00,Matchweek 4,Manchester City,Fulham,Home,W,5,1,2.2,1.4,68.0,52899.0,Kyle Walker,4-2-3-1,4-3-3,Michael Oliver,6.0,4.0,14.8,0.0,1,1
4,2023,2023-09-16,Sat,15:00,Matchweek 5,Manchester City,West Ham,Away,W,3,1,3.6,0.9,68.0,62475.0,Kyle Walker,4-2-3-1,4-2-3-1,Andy Madley,29.0,13.0,16.4,1.0,0,0


In [1241]:
3 * 38 * 20 #total number of games

2280

In [1242]:
matches.shape

(2280, 25)

In [1243]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2280 entries, 0 to 2279
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   season         2280 non-null   int64  
 1   date           2280 non-null   object 
 2   day            2280 non-null   object 
 3   time           2280 non-null   object 
 4   round          2280 non-null   object 
 5   team           2280 non-null   object 
 6   opponent       2280 non-null   object 
 7   venue          2280 non-null   object 
 8   result         2280 non-null   object 
 9   gf             2280 non-null   int64  
 10  ga             2280 non-null   int64  
 11  xg             2280 non-null   float64
 12  xga            2280 non-null   float64
 13  poss           2280 non-null   float64
 14  attendance     2278 non-null   float64
 15  captain        2280 non-null   object 
 16  formation      2280 non-null   object 
 17  opp formation  2280 non-null   object 
 18  referee 

In [1244]:
matches['team'].value_counts()

Manchester City             114
West Ham United             114
Brentford                   114
Everton                     114
Wolverhampton Wanderers     114
Arsenal                     114
Brighton and Hove Albion    114
Crystal Palace              114
Manchester United           114
Newcastle United            114
Chelsea                     114
Tottenham Hotspur           114
Aston Villa                 114
Liverpool                   114
Burnley                      76
Southampton                  76
Leeds United                 76
Leicester City               76
Fulham                       76
Nottingham Forest            76
Bournemouth                  76
Sheffield United             38
Luton Town                   38
Watford                      38
Norwich City                 38
Name: team, dtype: int64

Converting non numeric dtypes to numeric 

In [1245]:
matches["date"] = pd.to_datetime(matches["date"])

In [1246]:
matches["target"] = (matches["result"] == "W").astype("int")

In [1247]:
matches["venue_code"] = (matches["venue"] == "Home").astype("int")

In [1248]:
matches["opp_code"] = (matches["opponent"]).astype("category").cat.codes

In [1249]:
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")

In [1250]:
matches["day_code"] = matches["date"].dt.dayofweek

In [1251]:
matches["ref_code"] = matches["referee"].astype("category").cat.codes

In [1252]:
grouped_matches = matches.groupby("team")

In [1253]:
grouped_matches.head()

,season,date,day,time,round,team,opponent,venue,result,gf,ga,xg,xga,poss,attendance,captain,formation,opp formation,referee,sh,sot,dist,fk,pk,pkatt,target,venue_code,opp_code,hour,day_code,ref_code
0,2023,2023-08-11,Fri,20:00,Matchweek 1,Manchester City,Burnley,Away,W,3,0,1.9,0.3,65.0,21572.0,Kevin De Bruyne,4-2-3-1,5-4-1,Craig Pawson,17.0,8.0,13.9,0.0,0,0,1,0,5,20,4,4
1,2023,2023-08-19,Sat,20:00,Matchweek 2,Manchester City,Newcastle Utd,Home,W,1,0,1.0,0.3,59.0,53419.0,Kyle Walker,4-2-3-1,4-3-3,Robert Jones,14.0,4.0,17.9,0.0,0,0,1,1,16,20,5,23
2,2023,2023-08-27,Sun,14:00,Matchweek 3,Manchester City,Sheffield Utd,Away,W,2,1,3.5,0.7,79.0,31336.0,Kyle Walker,4-2-3-1,3-5-2,Jarred Gillett,29.0,9.0,17.3,2.0,0,1,1,0,19,14,6,9
3,2023,2023-09-02,Sat,15:00,Matchweek 4,Manchester City,Fulham,Home,W,5,1,2.2,1.4,68.0,52899.0,Kyle Walker,4-2-3-1,4-3-3,Michael Oliver,6.0,4.0,14.8,0.0,1,1,1,1,9,15,5,17
4,2023,2023-09-16,Sat,15:00,Matchweek 5,Manchester City,West Ham,Away,W,3,1,3.6,0.9,68.0,62475.0,Kyle Walker,4-2-3-1,4-2-3-1,Andy Madley,29.0,13.0,16.4,1.0,0,0,1,0,23,15,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2242,2021,2021-08-14,Sat,17:30,Matchweek 1,Norwich City,Liverpool,Home,L,0,3,1.4,1.6,50.0,27023.0,Grant Hanley,4-3-3,4-3-3,Andre Marriner,14.0,3.0,17.1,0.0,0,0,0,1,12,17,5,0
2243,2021,2021-08-21,Sat,15:00,Matchweek 2,Norwich City,Manchester City,Away,L,0,5,0.1,2.6,33.0,51437.0,Grant Hanley,4-3-3,4-3-3,Graham Scott,1.0,0.0,16.8,0.0,0,0,0,0,14,15,5,8
2244,2021,2021-08-28,Sat,15:00,Matchweek 3,Norwich City,Leicester City,Home,L,1,2,1.5,1.5,45.0,26765.0,Grant Hanley,4-3-3,4-4-2,Robert Jones,13.0,3.0,22.6,1.0,1,1,0,1,11,15,5,23
2245,2021,2021-09-11,Sat,15:00,Matchweek 4,Norwich City,Arsenal,Away,L,0,1,0.7,2.7,49.0,58000.0,Grant Hanley,4-3-3,4-2-3-1,Michael Oliver,10.0,1.0,19.9,1.0,0,0,0,0,0,15,5,17


In [1254]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators= 50, min_samples_split= 10, random_state= 1)

train = matches[matches["date"] < '2023-08-01']
test = matches[matches["date"] > '2023-08-01']

In [1255]:

predictors = ["venue_code", "opp_code", "hour", "ref_code", "day_code"]

xtrain = train[predictors]
ytrain = train["target"]

In [1256]:
xtest = test[predictors]
ytest = test["target"]

In [1257]:
rf.fit(xtrain, ytrain)

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [1258]:
ypred = rf.predict(xtest)

In [1259]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(ypred, ytest)

In [1260]:
accuracy

0.6026315789473684

## Retraining the model to improve the accuracy

In [1261]:
matches['formation_code'] = matches['formation'].astype("category").cat.codes

In [1262]:
matches["comb_formation_code"] = ((matches["formation"] + matches["opp formation"]).astype("category").cat.codes)

In [1263]:
matches["captain_code"] = matches["captain"].astype("category").cat.codes

In [1264]:
grouped_matches = matches.groupby("team")

In [1265]:
group = grouped_matches.get_group("Manchester City").sort_values("date")

In [1266]:
group

,season,date,day,time,round,team,opponent,venue,result,gf,ga,xg,xga,poss,attendance,captain,formation,opp formation,referee,sh,sot,dist,fk,pk,pkatt,target,venue_code,opp_code,hour,day_code,ref_code,formation_code,comb_formation_code,captain_code
1520,2021,2021-08-15,Sun,16:30,Matchweek 1,Manchester City,Tottenham,Away,L,0,1,1.8,1.0,65.0,58262.0,Fernandinho,4-3-3,4-3-3,Anthony Taylor,18.0,4.0,17.3,1.0,0,0,0,0,21,16,6,2,15,134,39
1521,2021,2021-08-21,Sat,15:00,Matchweek 2,Manchester City,Norwich City,Home,W,5,0,2.6,0.1,67.0,51437.0,İlkay Gündoğan,4-3-3,4-3-3,Graham Scott,16.0,4.0,18.5,1.0,0,0,1,1,17,15,5,8,15,134,133
1522,2021,2021-08-28,Sat,12:30,Matchweek 3,Manchester City,Arsenal,Home,W,5,0,4.4,0.2,80.0,52276.0,İlkay Gündoğan,4-3-3,5-4-1,Martin Atkinson,25.0,10.0,14.8,0.0,0,0,1,1,0,12,5,15,15,139,133
1523,2021,2021-09-11,Sat,15:00,Matchweek 4,Manchester City,Leicester City,Away,W,1,0,2.8,0.6,61.0,32087.0,İlkay Gündoğan,4-3-3,4-2-3-1,Paul Tierney,25.0,8.0,14.3,0.0,0,0,1,0,11,15,5,20,15,131,133
1524,2021,2021-09-18,Sat,15:00,Matchweek 5,Manchester City,Southampton,Home,D,0,0,1.0,0.4,64.0,52698.0,Fernandinho,4-3-3,4-4-2,Jonathan Moss,16.0,1.0,16.4,1.0,0,0,0,1,20,15,5,11,15,136,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,2023,2024-04-28,Sun,16:30,Matchweek 35,Manchester City,Nott'ham Forest,Away,W,2,0,0.9,1.9,66.0,29677.0,Kyle Walker,4-1-4-1,5-4-1,Simon Hooper,11.0,5.0,18.7,1.0,0,0,1,0,18,16,6,27,9,81,74
34,2023,2024-05-04,Sat,17:30,Matchweek 36,Manchester City,Wolves,Home,W,5,1,3.7,0.3,58.0,52439.0,Kyle Walker,4-2-3-1,3-5-2,Craig Pawson,18.0,10.0,15.3,0.0,2,2,1,1,24,17,5,4,11,92,74
35,2023,2024-05-11,Sat,12:30,Matchweek 37,Manchester City,Fulham,Away,W,4,0,3.2,0.1,64.0,24418.0,Kevin De Bruyne,4-2-3-1,4-2-3-1,Anthony Taylor,15.0,8.0,17.6,0.0,1,1,1,0,9,12,5,2,11,96,71
36,2023,2024-05-14,Tue,20:00,Matchweek 34,Manchester City,Tottenham,Away,W,2,0,2.4,1.7,47.0,61065.0,Kyle Walker,4-2-3-1,4-3-1-2,Chris Kavanagh,7.0,4.0,14.5,0.0,1,1,1,0,21,20,1,3,11,97,74


In [1267]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2280 entries, 0 to 2279
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   season               2280 non-null   int64         
 1   date                 2280 non-null   datetime64[ns]
 2   day                  2280 non-null   object        
 3   time                 2280 non-null   object        
 4   round                2280 non-null   object        
 5   team                 2280 non-null   object        
 6   opponent             2280 non-null   object        
 7   venue                2280 non-null   object        
 8   result               2280 non-null   object        
 9   gf                   2280 non-null   int64         
 10  ga                   2280 non-null   int64         
 11  xg                   2280 non-null   float64       
 12  xga                  2280 non-null   float64       
 13  poss                 2280 non-nul

Finding rolling averages(averages of previous n datas ) 

In [1268]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(5, closed = 'left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset= new_cols)
    return group

In [1269]:
cols = ["gf", "ga", "xg", "xga", "sh", "sot", "dist", "fk", "pk", "pkatt", "poss"]
new_cols = [f"{c}_rolling" for c in cols]

rolling_averages(group, cols, new_cols)

,season,date,day,time,round,team,opponent,venue,result,gf,ga,xg,xga,poss,attendance,captain,formation,opp formation,referee,sh,sot,dist,fk,pk,pkatt,target,venue_code,opp_code,hour,day_code,ref_code,formation_code,comb_formation_code,captain_code,gf_rolling,ga_rolling,xg_rolling,xga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling,poss_rolling
1525,2021,2021-09-25,Sat,12:30,Matchweek 6,Manchester City,Chelsea,Away,W,1,0,1.7,0.2,59.0,40036.0,Rúben Dias,4-3-3,3-5-2,Michael Oliver,15.0,3.0,17.1,0.0,0,0,1,0,6,12,5,17,15,127,110,2.2,0.2,2.52,0.46,20.0,5.4,16.26,0.6,0.0,0.0,67.4
1526,2021,2021-10-03,Sun,16:30,Matchweek 7,Manchester City,Liverpool,Away,D,2,2,1.2,0.9,51.0,53102.0,Rúben Dias,4-3-3,4-3-3,Paul Tierney,12.0,3.0,17.0,1.0,0,0,0,0,12,16,6,20,15,134,110,2.4,0.0,2.50,0.30,19.4,5.2,16.22,0.4,0.0,0.0,66.2
1527,2021,2021-10-16,Sat,15:00,Matchweek 8,Manchester City,Burnley,Home,W,2,0,2.1,0.9,70.0,52711.0,Kevin De Bruyne,4-3-3,4-4-2,Martin Atkinson,15.0,6.0,14.4,0.0,0,0,1,1,5,15,5,15,15,136,71,1.8,0.4,2.22,0.46,18.6,5.0,15.92,0.4,0.0,0.0,63.0
1528,2021,2021-10-23,Sat,17:30,Matchweek 9,Manchester City,Brighton,Away,W,4,1,3.8,1.3,52.0,31215.0,İlkay Gündoğan,4-3-3,3-5-2,Kevin Friend,23.0,12.0,16.0,1.0,0,0,1,0,4,17,5,13,15,127,133,1.2,0.4,1.76,0.60,16.6,4.2,15.84,0.4,0.0,0.0,61.0
1529,2021,2021-10-30,Sat,15:00,Matchweek 10,Manchester City,Crystal Palace,Home,L,0,2,0.7,0.7,68.0,53014.0,Rúben Dias,4-3-3,4-3-3,Andre Marriner,14.0,3.0,17.7,0.0,0,0,0,1,7,15,5,0,15,134,110,1.8,0.6,1.96,0.74,16.2,5.0,16.18,0.6,0.0,0.0,59.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,2023,2024-04-28,Sun,16:30,Matchweek 35,Manchester City,Nott'ham Forest,Away,W,2,0,0.9,1.9,66.0,29677.0,Kyle Walker,4-1-4-1,5-4-1,Simon Hooper,11.0,5.0,18.7,1.0,0,0,1,0,18,16,6,27,9,81,74,3.4,0.8,2.38,0.74,21.0,7.6,16.02,0.6,0.2,0.2,70.2
34,2023,2024-05-04,Sat,17:30,Matchweek 36,Manchester City,Wolves,Home,W,5,1,3.7,0.3,58.0,52439.0,Kyle Walker,4-2-3-1,3-5-2,Craig Pawson,18.0,10.0,15.3,0.0,2,2,1,1,24,17,5,4,11,92,74,3.8,0.8,2.36,0.98,20.8,8.4,16.92,0.8,0.2,0.2,69.0
35,2023,2024-05-11,Sat,12:30,Matchweek 37,Manchester City,Fulham,Away,W,4,0,3.2,0.1,64.0,24418.0,Kevin De Bruyne,4-2-3-1,4-2-3-1,Anthony Taylor,15.0,8.0,17.6,0.0,1,1,1,0,9,12,5,2,11,96,71,4.0,0.8,2.42,0.88,19.4,8.2,16.80,0.4,0.6,0.6,67.2
36,2023,2024-05-14,Tue,20:00,Matchweek 34,Manchester City,Tottenham,Away,W,2,0,2.4,1.7,47.0,61065.0,Kyle Walker,4-2-3-1,4-3-1-2,Chris Kavanagh,7.0,4.0,14.5,0.0,1,1,1,0,21,20,1,3,11,97,74,4.0,0.4,2.68,0.68,18.8,8.2,17.10,0.4,0.8,0.8,65.0


In [1270]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))

In [1271]:
matches_rolling.head()

season       date  day  ... pk_rolling pkatt_rolling poss_rolling
team                                  ...                                      
Arsenal 1677    2021 2021-09-26  Sun  ...        0.0           0.0         44.8
        1678    2021 2021-10-02  Sat  ...        0.0           0.0         41.2
        1679    2021 2021-10-18  Mon  ...        0.0           0.0         42.6
        1680    2021 2021-10-22  Fri  ...        0.0           0.0         49.4
        1681    2021 2021-10-30  Sat  ...        0.0           0.2         49.8

[5 rows x 45 columns]

In [1272]:
matches_rolling = matches_rolling.droplevel('team')

In [1273]:
matches_rolling.head()

,season,date,day,time,round,team,opponent,venue,result,gf,ga,xg,xga,poss,attendance,captain,formation,opp formation,referee,sh,sot,dist,fk,pk,pkatt,target,venue_code,opp_code,hour,day_code,ref_code,formation_code,comb_formation_code,captain_code,gf_rolling,ga_rolling,xg_rolling,xga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling,poss_rolling
1677,2021,2021-09-26,Sun,16:30,Matchweek 6,Arsenal,Tottenham,Home,W,3,1,1.1,1.0,46.0,59919.0,Pierre-Emerick Aubameyang,4-2-3-1,4-3-3,Craig Pawson,12.0,7.0,17.7,0.0,0,0,1,1,21,16,6,4,11,99,103,0.4,1.8,1.12,2.08,14.4,3.2,16.42,0.6,0.0,0.0,44.8
1678,2021,2021-10-02,Sat,17:30,Matchweek 7,Arsenal,Brighton,Away,D,0,0,0.4,1.1,42.0,31266.0,Pierre-Emerick Aubameyang,4-2-3-1,3-5-2,Jonathan Moss,8.0,2.0,21.1,1.0,0,0,0,0,4,17,5,11,11,92,103,1.0,1.6,1.08,2.04,12.4,3.8,16.16,0.4,0.0,0.0,41.2
1679,2021,2021-10-18,Mon,20:00,Matchweek 8,Arsenal,Crystal Palace,Home,D,2,2,1.7,0.7,54.0,59475.0,Pierre-Emerick Aubameyang,4-1-4-1,4-2-3-1,Mike Dean,17.0,6.0,17.6,1.0,0,0,0,1,7,20,0,19,9,76,103,1.0,1.2,1.10,1.64,12.8,3.6,16.64,0.6,0.0,0.0,42.6
1680,2021,2021-10-22,Fri,20:00,Matchweek 9,Arsenal,Aston Villa,Home,W,3,1,2.7,1.4,53.0,59496.0,Pierre-Emerick Aubameyang,4-2-3-1,3-4-1-2,Craig Pawson,21.0,8.0,17.3,1.0,0,1,1,1,1,20,4,4,11,89,103,1.4,0.6,1.40,0.90,16.0,4.8,18.80,0.8,0.0,0.0,49.4
1681,2021,2021-10-30,Sat,12:30,Matchweek 10,Arsenal,Leicester City,Away,W,2,0,0.7,1.3,36.0,32209.0,Pierre-Emerick Aubameyang,4-4-1-1,3-4-1-2,Michael Oliver,9.0,5.0,16.7,0.0,0,0,1,0,11,12,5,17,16,142,103,1.8,0.8,1.40,1.04,14.2,5.2,18.88,0.8,0.0,0.2,49.8


In [1274]:
matches_rolling.index = range(matches_rolling.shape[0])

In [1275]:
matches_rolling.head()


,season,date,day,time,round,team,opponent,venue,result,gf,ga,xg,xga,poss,attendance,captain,formation,opp formation,referee,sh,sot,dist,fk,pk,pkatt,target,venue_code,opp_code,hour,day_code,ref_code,formation_code,comb_formation_code,captain_code,gf_rolling,ga_rolling,xg_rolling,xga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling,poss_rolling
0,2021,2021-09-26,Sun,16:30,Matchweek 6,Arsenal,Tottenham,Home,W,3,1,1.1,1.0,46.0,59919.0,Pierre-Emerick Aubameyang,4-2-3-1,4-3-3,Craig Pawson,12.0,7.0,17.7,0.0,0,0,1,1,21,16,6,4,11,99,103,0.4,1.8,1.12,2.08,14.4,3.2,16.42,0.6,0.0,0.0,44.8
1,2021,2021-10-02,Sat,17:30,Matchweek 7,Arsenal,Brighton,Away,D,0,0,0.4,1.1,42.0,31266.0,Pierre-Emerick Aubameyang,4-2-3-1,3-5-2,Jonathan Moss,8.0,2.0,21.1,1.0,0,0,0,0,4,17,5,11,11,92,103,1.0,1.6,1.08,2.04,12.4,3.8,16.16,0.4,0.0,0.0,41.2
2,2021,2021-10-18,Mon,20:00,Matchweek 8,Arsenal,Crystal Palace,Home,D,2,2,1.7,0.7,54.0,59475.0,Pierre-Emerick Aubameyang,4-1-4-1,4-2-3-1,Mike Dean,17.0,6.0,17.6,1.0,0,0,0,1,7,20,0,19,9,76,103,1.0,1.2,1.10,1.64,12.8,3.6,16.64,0.6,0.0,0.0,42.6
3,2021,2021-10-22,Fri,20:00,Matchweek 9,Arsenal,Aston Villa,Home,W,3,1,2.7,1.4,53.0,59496.0,Pierre-Emerick Aubameyang,4-2-3-1,3-4-1-2,Craig Pawson,21.0,8.0,17.3,1.0,0,1,1,1,1,20,4,4,11,89,103,1.4,0.6,1.40,0.90,16.0,4.8,18.80,0.8,0.0,0.0,49.4
4,2021,2021-10-30,Sat,12:30,Matchweek 10,Arsenal,Leicester City,Away,W,2,0,0.7,1.3,36.0,32209.0,Pierre-Emerick Aubameyang,4-4-1-1,3-4-1-2,Michael Oliver,9.0,5.0,16.7,0.0,0,0,1,0,11,12,5,17,16,142,103,1.8,0.8,1.40,1.04,14.2,5.2,18.88,0.8,0.0,0.2,49.8


In [1276]:
rf_new = RandomForestClassifier(n_estimators= 50, min_samples_split= 10, random_state= 1)

train = matches_rolling[matches_rolling["date"] < '2024-01-01']
test = matches_rolling[matches_rolling["date"] > '2024-01-01']
predictors = ["venue_code", "opp_code", "hour", "ref_code", "day_code", "captain_code", "formation_code", "comb_formation_code"] + new_cols

xtrain = train[predictors]
ytrain = train["target"]

rf_new.fit(xtrain, ytrain)

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [1277]:
xtest = test[predictors]
ytest = test["target"]

In [1278]:
ypred = rf_new.predict(xtest)

In [1279]:
accuracy = accuracy_score(ytest, ypred)


In [1280]:
accuracy

0.7021857923497268

In [1281]:
combined = pd.DataFrame(dict(actual=ytest, predicted=ypred), index=test.index)

In [1282]:
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)

In [1283]:
combined.sample(10)

,actual,predicted,date,team,opponent,result
488,0,0,2024-01-02,Brighton and Hove Albion,West Ham,D
673,0,0,2024-02-17,Chelsea,Manchester City,D
1361,1,0,2024-04-06,Manchester City,Crystal Palace,W
1680,0,0,2024-03-10,Nottingham Forest,Brighton,L
1716,0,0,2024-04-07,Sheffield United,Chelsea,D
490,0,0,2024-01-30,Brighton and Hove Albion,Luton Town,L
1476,1,0,2024-05-15,Manchester United,Newcastle Utd,W
388,0,0,2024-03-16,Brentford,Burnley,L
1462,1,1,2024-02-04,Manchester United,West Ham,W
1471,0,0,2024-04-13,Manchester United,Bournemouth,D
